# ENV

In [6]:
%%capture
!pip install selenium
!pip install -y chromium-chromedriver
!pip install unidecode
!pip install langdetect
!pip install thefuzz

In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from IPython.display import clear_output
import time

from tqdm.auto import tqdm
import pandas as pd
import gc

In [8]:
service = Service()

chrome_options = Options()
chrome_options.page_load_strategy = 'eager'
chrome_options.add_argument("--incognito")

chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument("--disable-infobars")
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-notifications")
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("--ignore-certificate-errors")
chrome_options.add_argument("disable-infobars")

chrome_options.binary_location='/content/chromedriver.exe'
prefs = {'profile.default_content_setting_values': {'cookies': 2, 'images': 2, 'javascript': 2,
                            'plugins': 2, 'popups': 2, 'geolocation': 2,
                            'notifications': 2, 'auto_select_certificate': 2, 'fullscreen': 2,
                            'mouselock': 2, 'mixed_script': 2, 'media_stream': 2,
                            'media_stream_mic': 2, 'media_stream_camera': 2, 'protocol_handlers': 2,
                            'ppapi_broker': 2, 'automatic_downloads': 2, 'midi_sysex': 2,
                            'push_messaging': 2, 'ssl_cert_decisions': 2, 'metro_switch_to_desktop': 2,
                            'protected_media_identifier': 2, 'app_banner': 2, 'site_engagement': 2,
                            'durable_storage': 2}}
chrome_options.add_experimental_option('prefs', prefs)

# Selenium Crawl

In [ ]:
profile = pd.DataFrame(
    {'Name': [], 'Link': [],'Page link':[]})

paper= pd.DataFrame(
    {'Title': [], 'Link': [], 'Check':[]})

In [ ]:
%%time
def get_profile_link(link):
    cur_row=0
    for i,row in profile.iterrows():
        if row['Page link']!='nan':
            cur_row=i
            link=row['Page link']
        else:
            break
    driver = webdriver.Chrome(service=service,options=chrome_options)
    driver.get(link)
    next_button = driver.find_element(By.XPATH, '//*[@id="gsc_authors_bottom_pag"]/div/button[2]')
    while 1:
        if cur_row>=1 and cur_row%1000==0:
            # time.sleep(30)
            cur_url=driver.current_url
            driver = webdriver.Chrome(service=service,options=chrome_options)
            driver.get(cur_url)

        print(driver.current_url)
        items=driver.find_elements(By.XPATH,'//div[@class="gs_ai gs_scl gs_ai_chpr"]')
        for i in items:
            tmp=i.find_element(By.XPATH,'.//h3[@class="gs_ai_name"]/a')
            profile.loc[cur_row,'Name']=tmp.text
            profile.loc[cur_row,'Link']=tmp.get_attribute('href')
            profile.loc[cur_row,'Citation']=i.find_element(By.XPATH,'.//div[@class="gs_ai_aff"]').text
            profile.loc[cur_row,'School']=i.find_element(By.XPATH,'.//div[@class="gs_ai_cby"]').text
            profile.loc[cur_row,'Page link']=driver.current_url
            profile.loc[cur_row,'Check']='0'
            cur_row=cur_row+1

        WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//div[@id="gsc_authors_bottom_pag"]/div/button[2]')))
        next_button = driver.find_element(By.XPATH, '//div[@id="gsc_authors_bottom_pag"]/div/button[2]')
        if next_button.is_enabled()==False:
            driver.close()
            break
        else:
            print(5)
            next_button.click()

# get_profile_link('https://scholar.google.com.au/citations?hl=en&view_op=search_authors&mauthors=VNU-University+of+Engineering+and+Technology&btnG=')
# get_profile_link('https://scholar.google.com.au/citations?view_op=view_org&hl=en&org=13319503632786364699')


get_profile_link("https://scholar.google.com/citations?view_op=view_org&hl=en&org=8515235176732148308") #totronto
# get_profile_link("https://scholar.google.com/citations?view_op=view_org&hl=en&org=11816294095661060495") #University of California Berkeley
# get_profile_link('https://scholar.google.com/citations?view_op=view_org&hl=en&org=8539678734835078480') #stanford
# get_profile_link('https://scholar.google.com/citations?view_op=view_org&hl=en&org=273144080909810983') #oxford

In [ ]:
profile.drop_duplicates(inplace=True)
profile.reset_index(drop=True,inplace=True)
profile.to_excel('Profile.xlsx',index=False)

In [10]:
import pandas as pd

paper= pd.DataFrame(
    {'Title': [], 'Link': [], 'Check':[]})

profile=pd.read_excel('Teacher_UET.xlsx')
profile = profile.fillna('')
profile.astype(str)
profile.head()

,Bộ môn,Tên,Chức danh,Hướng nghiên cứu,Chủ đề khóa luận,Liên hệ,HomePage,Link
0,Công nghệ phần mềm,Phạm Ngọc Hùng,PGS.TS,Phương pháp hình thức cho phát triển PM\nKiểm ...,Các phương pháp đặc tả và kiểm chứng cho các h...,hungpn@vnu.edu.vn,,https://scholar.google.com/citations?hl=en&use...
1,Công nghệ phần mềm,Võ Đình Hiếu,TS,"Kiến trúc phần mềm (microservices, SOA, cloud ...",,hieuvd@vnu.edu.vn,,https://scholar.google.com/citations?hl=en&use...
2,Công nghệ phần mềm,Đặng Đức Hạnh,TS,Kỹ nghệ mô hình phần mềm (model transformation...,Kỹ nghệ mô hình phần mềm (model transformation...,hanhdd@vnu.edu.vn,,https://scholar.google.com/citations?hl=en&use...
3,Công nghệ phần mềm,Vũ Thị Hồng Nhạn,TS,"Khai phá dữ liệu, học máy, nhận dạng mẫu\nTối ...",,vthnhan@vnu.edu.vn,,https://scholar.google.com/citations?hl=en&use...
4,Công nghệ phần mềm,Trần Hoàng Việt,TS,Phát triển ứng dụng trên nền web và di động\nĐ...,,thv@vnu.edu.vn,,https://scholar.google.com/citations?hl=en&use...


In [75]:
import re
from unidecode import unidecode

def clean_text(text, mode=1):
    if mode==1:
        # keep -, white space, spilt by other punctuation
        result=re.split(r'[^\w\s-]',text)
    if mode==2:
        # keep white space, spilt by other punctuation
        result=re.split(r'[^\w\s]',text)
    else:
        # keep word only
        result=re.split(r'[^\w]',text)
    result=[' '.join(s.split()).lower() for s in result if s!='']
    if mode!=1:
        result=[unidecode(word) for word in result]
    return result

# clean_text("   web'\(trang web - 1090 !); very  \n good",mode=2)
clean_text('Duc-Trong Le',mode=3)
# clean_text("tôi \n yêu \n em",mode=2)

['duc', 'trong', 'le']

In [32]:
# get paper link
from unidecode import unidecode

cur_row=len(paper)
driver = webdriver.Chrome(service=service,options=chrome_options)

for i in tqdm(range(len(profile))):
    if profile.loc[i,'Link']=='':
        continue
    if i%50==49:
        driver.close()
        driver = webdriver.Chrome(service=service,options=chrome_options)
    driver.get(profile.loc[i,'Link'])
    showmore_button=driver.find_element(By.XPATH, '//button[@id="gsc_bpf_more"]')
    while showmore_button.is_enabled():
        showmore_button.click()


    current_keywords=clean_text(profile.loc[i,'Hướng nghiên cứu'])
    new_keywords=driver.find_elements(By.XPATH,'//div[@id="gsc_prf_int"]/a')
    new_keywords=[' '.join(clean_text(word.text)) for word in new_keywords]
    new_keywords=[word for word in new_keywords if word not in current_keywords]

    current_keyword=clean_text(profile.loc[i,'Hướng nghiên cứu'])
    profile.loc[i,'Hướng nghiên cứu']=', '.join(new_keywords+current_keywords)

    new_homepage=driver.find_elements(By.XPATH, '//div[@id="gsc_prf_ivh"]/a')
    if len(new_homepage)>0:
        curent_homepage =profile.loc[i,'HomePage'].split('\n')
        new_homepage=new_homepage[0].get_attribute('href')
        if new_homepage not in curent_homepage:
            if profile.loc[i,'HomePage']!='':
                profile.loc[i,'HomePage']=profile.loc[i,'HomePage']+'\n'+new_homepage
            else:
                profile.loc[i,'HomePage']=new_homepage

    title=driver.find_elements(By.XPATH, '//td[@class="gsc_a_t"]/a')
    profile.loc[i,'Number of paper']=len(title)
    all_paper=''
    for j in title:
        all_paper=all_paper+j.text+'\n'
        titles=j.text.split('=')
        paper.loc[cur_row,'Title']=titles[0]
        paper.loc[cur_row,'Clean_Title']=unidecode(' '.join(clean_text(titles[0],mode=2)))
        paper.loc[cur_row,'Link']=j.get_attribute('href')
        paper.loc[cur_row,'Check']='0'

        cur_row=cur_row+1

        if len(titles)>1:
            paper.loc[cur_row]=paper.loc[cur_row-1]
            paper.loc[cur_row,'Title']=titles[1]
            paper.loc[cur_row,'Clean_Title']=unidecode(' '.join(clean_text(titles[1],mode=2)))

            cur_row=cur_row+1

    profile.loc[i,'Papers']=all_paper
    profile.loc[i,'Check']='1'
    # clear_output(wait=True)
    # display(profile.loc[[i]])
driver.close()

profile.to_excel('Teacher_UET_1.xlsx',index=False)

  0%|          | 0/119 [00:00<?, ?it/s]

In [13]:
# drop duplicate
from thefuzz import fuzz

paper.drop_duplicates(subset=['Clean_Title','Link'],inplace=True)
paper = paper.iloc[paper['Title'].apply(lambda x: ''.join(sorted(x))).argsort()]
paper.reset_index(drop=True,inplace=True)
duplicate_id=[]
for i,row in tqdm(paper.iterrows(),total=len(paper)):
    if i==0:
        continue
    if fuzz.ratio(paper.loc[i, 'Clean_Title'], paper.loc[i-1, 'Clean_Title'])>90:
        duplicate_id.append(i)

paper=paper[~paper.index.isin(duplicate_id)]
paper.reset_index(drop=True,inplace=True)
paper.to_excel('Paper_UET.xlsx',index=False)

  0%|          | 0/2204 [00:00<?, ?it/s]

In [17]:
# get paper infor

pd.options.mode.copy_on_write = False
driver = webdriver.Chrome(service=service,options=chrome_options)
for i in tqdm(range(len(paper))):
    # if paper.loc[i,'Check']=='1':
        # continue
    if i%50==49:
        driver.close()
        driver = webdriver.Chrome(service=service,options=chrome_options)
    driver.get(paper.loc[i,'Link'])
    value_list=driver.find_elements(By.XPATH, '//div[@class="gsc_oci_value"]')
    field_list=driver.find_elements(By.XPATH, '//div[@class="gsc_oci_field"]')
    for field,value in zip(field_list, value_list):
        if field.text!='Total citations':
            text=value.text
        else:
            text=value.find_element(By.XPATH,'.//a').text

        paper.loc[i,field.text]=text

    files=driver.find_elements(By.XPATH, '//div[@class="gsc_oci_title_ggi"]/a')
    if len(files)>0:
        pdf=''
        for f in files:
            pdf=pdf+f.get_attribute('href')+'\n'
        paper.loc[i,'Files']=pdf

    paper.loc[i,'Check']='1'
    # clear_output(wait=True)
    # display(paper.loc[[i]])
driver.close()
paper.to_excel('Paper_UET.xlsx',index=False)

  0%|          | 0/1784 [00:00<?, ?it/s]

In [18]:
paper.columns

Index(['Title', 'Link', 'Check', 'Clean_Title', 'Authors', 'Publication date',
       'Publisher', 'Journal', 'Volume', 'Issue', 'Pages', 'Description',
       'Scholar articles', 'Files', 'Total citations', 'Inventors',
       'Patent office', 'Patent number', 'Application number', 'Institution',
       'Conference', 'Source', 'Book', 'Report number'],
      dtype='object')

In [ ]:
from langdetect import detect

def non_english_detect(text,thresh_hold=0.6):
    tmp=re.sub(r'[^\w\s]+', '', text).lower()
    white_space=tmp.count(" ")
    latin_count=len(re.findall(r'[a-zA-Z]', tmp))

    if detect(tmp)=='vi':
        # print(2)
        return True
    elif latin_count/(len(tmp)-white_space)<=thresh_hold:
        # print(3)
        return True
    else:
        return False

not_en=[]

for idx, row in tqdm(paper.iterrows(),total=len(paper)):
    if non_english_detect(row['Title'])==True:
        not_en.append(idx)

print(len(not_en))
for idx,row in paper.loc[not_en].iterrows():
    print(row['Title'])

# paper=paper[~paper.index.isin(not_en)]

  0%|          | 0/1936 [00:00<?, ?it/s]

210
Mô hình cơ sở dữ liệu quan hệ và mô hình cơ sở dữ liệu hướng đối tượng (Báo cáo tổng hợp Đề tài nghiên cứu khoa học đo Khoa quản lý cấp DI 1QG Hà nội): Đề tài NCKH. QC. 04.06
XÂY DỰNG BẢN ĐỒ PHÂN VÙNG KHẢ NĂNG HẠN HÁN TRÊN ĐẤT TRỒNG LÚA DỰA VÀO CHỈ SỐ CHUẨN HÓA GIÁNG THỦY TẠI HUYỆN HÒA VANG, THÀNH PHỐ ĐÀ NẴNG: Đất trồng lúa, GIS, hạn hán, huyện Hòa …
Nhận dạng tiếng Việt trên các thiết bị cầm tay (Báo cáo tổng hợp đề tài nghiên cứu khoa học cấp Đại học Quốc gia do Trường Đại học Công nghệ quản lý): Đề tài NCKH QC. 06.23
Ứng dụng thực tại trộn trong đào tạo điện tử (Báo cáo tổng hợp đề tài nghiên cứu khoa học cấp Đại học Quốc gia do Khoa Công nghệ quản lý): Đề tài NCKH. QC. 04.03
Hiệu quả kinh tế của ứng dụng công nghệ cao trong sản xuất kinh doanh ở các hợp tác xã trồng cây ăn quả trên địa bàn huyện Mai Sơn, tỉnh Sơn La
Đặc điểm sinh trưởng và thức ăn tự nhiên của Ngao dầu Meretrix meretrix Linne ở cửa sông Hồng [Growth characteristic and natural food of clam Meretrix meretrix in t

In [15]:
driver = webdriver.Chrome(service=service,options=chrome_options)
driver.get('https://scholar.google.com/citations?view_op=view_citation&hl=en&user=KDJCrhMAAAAJ&pagesize=80&citation_for_view=KDJCrhMAAAAJ:Y0pCki6q_DkC')

In [16]:
value_list=driver.find_elements(By.XPATH, '//div[@class="gsc_oci_value"]')
field_list=driver.find_elements(By.XPATH, '//div[@class="gsc_oci_field"]')
for field,value in zip(field_list, value_list):
    print(field.text,value.text)

Authors Phuong Hanh Du, Hai Chau Nguyen, Kim Khoa Nguyen, Ngoc Hoa Nguyen
Publication date 2018/12/6
Book Proceedings of the 9th International Symposium on Information and Communication Technology
Pages 456-462
Description Closeness centrality is an substantial metric used in large-scale network analysis, in particular social networks. Determining closeness centrality from a vertex to all other vertices in the graph is a high complexity problem. Prior work has a strong focuses on the algorithmic aspect of the problem, and little attention has been paid to the definition of the data structure supporting the implementation of the algorithm. Thus, we present in this paper an efficient algorithm to compute the closeness centrality of all nodes in a social network. Our algorithm is based on (i) an appropriate data structure for increasing the cache hit rate, and then reducing amount of time accessing the main memory for the graph data, and (ii) an efficient and parallel complete BFS search 

# Post Process

In [ ]:
import pandas as pd

teacher=pd.read_excel('Teacher_UET.xlsx')
teacher = teacher.fillna('')
teacher.astype(str)

paper=pd.read_excel('Paper_UET.xlsx')
paper = paper.fillna('')
paper.astype(str)

In [130]:
# paper[paper['Description'].map(lambda x: len(x)>=100)]
paper=paper[paper['Description'].map(lambda x: len(x)>=100)]
paper.reset_index(drop=True,inplace=True)
paper.to_excel('Paper_UET.xlsx',index=False)

In [ ]:
author_name=','.join(paper['Authors'].to_list())
for name in author_name.split(','):
    print(name)

In [101]:
# create name variation
def create_name_variation(name):
    word=clean_text(name,mode=3)
    variation=[]
    # Phạm Thị Kiều Trang -> pham thi kieu trang
    tmp=' '.join(word)
    variation.append(tmp)
    # ptk trang
    tmp=''
    for i in word[:-1]:
        tmp=tmp+i[0]
    tmp=tmp+' '+word[-1]
    variation.append(tmp)
    # tkt pham (trang kieu thi pham)
    tmp=''
    for i in word[1::]:
        tmp=i[0]+tmp
    tmp=tmp+' '+word[0]
    variation.append(tmp)

    # trang tk pham
    if len(word)>2:
        tmp=''
        for i in word[1:-1]:
            tmp=i[0]+tmp
        tmp=word[-1]+' '+tmp+' '+word[0]
        variation.append(tmp)
    return variation

# create_name_variation('Phạm Thị Kiều Trang')
for i in range(len(teacher)):
    teacher.loc[i,'Name variation']=','.join(create_name_variation(teacher.loc[i,'Tên']))

In [90]:
teacher.to_excel('Teacher_UET_1.xlsx',index=False)

In [104]:
hust=pd.read_excel('Thesis_HUST.xlsx')
hust=hust.fillna('')
hust.astype(str)

,Title,Link,Check,Authors,Advisor,Keywords,Issue Date,Publisher,Abstract,Description,URI,Appears in Collection,Other Titles,Editors
0,Đề xuất phương án nâng cấp hệ thống điều khiển...,https://dlib.hust.edu.vn/handle/HUST/14407,1,Trần Thị Như Trang,Hoàng Minh Sơn,Hệ thống điều khiển; Giám sát,2009.0,Trường Đại học Bách khoa Hà Nội,iới thiệu về mạng Profibus và cấu trúc mạng đi...,Luận văn Thạc sỹ - Ngành Điều khiển tự động,http://dlib.hust.edu.vn/handle/HUST/14407,Ths-Điện,,
1,"Nghiên cứu bộ điều khiển, bộ lọc tích cực xoay...",https://dlib.hust.edu.vn/handle/HUST/14408,1,Nguyễn Ngọc Hải,Lưu Hồng Việt,Bộ điều khiển; Bộ lọc,2009.0,Trường Đại học Bách khoa Hà Nội,,Luận văn Thạc sỹ - Ngành Điều khiển tự động,http://dlib.hust.edu.vn/handle/HUST/14408,Ths-Điện,,
2,Ứng dụng thuật toán logic mờ để điều khiển tốc...,https://dlib.hust.edu.vn/handle/HUST/14406,1,Trần Hồng Cầu,Phạm Thượng Hàn,Logic mờ; Tàu thuỷ; Ứng dụng,2009.0,Trường Đại học Bách khoa Hà Nội,Khái quát về động lực học và các nhiễu ảnh hưở...,Luận văn Thạc sỹ - Ngành Đo lường và các hệ th...,http://dlib.hust.edu.vn/handle/HUST/14406,Ths-Điện,,
3,Nghiên cứu hiệu quả của công nghệ truyền tải đ...,https://dlib.hust.edu.vn/handle/HUST/14470,1,Bùi Tiến Việt,Lã Văn Út,Hệ thống điện; Tải điện; Điện một chiều,2009.0,Trường Đại học Bách khoa Hà Nội,Tổng quan về công nghệ truyền tải điện một chi...,Luận văn (Thạc sỹ khoa học) - Ngành Hệ thống điện,http://dlib.hust.edu.vn/handle/HUST/14470,Ths-Điện,,
4,Phân tích ngắn mạch và nghiên cứu bảo vệ rơle ...,https://dlib.hust.edu.vn/handle/HUST/14471,1,Ngô Chí Trung,Trần Bách,Bảo vệ rơ le; Ngắn mạch; Nguồn điện phân tán,2009.0,Trường Đại học Bách khoa Hà Nội,Giới thiệu về nguồn điện phân tán. Tính toán n...,Luận văn (Thạc sỹ khoa học) - Ngành Mạng và Hệ...,http://dlib.hust.edu.vn/handle/HUST/14471,Ths-Điện,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17435,Phát triển phương pháp phủ tuyến tính để kiểm ...,https://dlib.hust.edu.vn/handle/HUST/11434,1,Phạm Văn Minh,Nguyễn Cảnh Quang\nNguyễn Thế Thắng,Điều khiển tuyến tính; Thiết kế bộ điều khiển,2017.0,Trường Đại học Bách Khoa Hà Nội,Tổng quan về hệ điều khiển bền vững với đối vớ...,,http://dlib.hust.edu.vn/handle/HUST/11434,Ts-Cơ khí,,
17436,Nghiên cứu điều khiển bộ biến đổi cộng hưởng b...,https://dlib.hust.edu.vn/handle/HUST/22896,1,Hoàng Trung Thông,Nguyễn Tùng Lâm\nNguyễn Kiên Trung,Bộ biến đổi cộng hưởng ba pha LLC; Bộ biến đổi...,2023.0,Trường đại học Bách Khoa Hà Nội,Tổng quan về hệ thống nguồn viễn thông; bộ biế...,,http://dlib.hust.edu.vn/handle/HUST/22896,Ts-Điện,,
17437,Ứng dụng điều khiển thích nghi hệ thiếu cơ cấu...,https://dlib.hust.edu.vn/handle/HUST/21407,1,Phạm Thị Hương Sen,Vũ Thị Thúy Nga\nPhan Xuân Minh,Xe tự hành ba bánh; Bộ điều khiển,2022.0,Trường đại học Bách Khoa Hà Nội,Tổng quan về hệ thiếu cơ cấu chấp hành và xe t...,,http://dlib.hust.edu.vn/handle/HUST/21407,Ts-Điện,,
17438,Nghiên cứu kỹ thuật đồng bộ và bù dịch tần Dop...,https://dlib.hust.edu.vn/handle/HUST/22052,1,Đỗ Đình Hưng,Nguyễn Quốc Khương\nHà Duyên Trung,Công nghệ OFDM; Kĩ thuật truyền thông,2023.0,Trường đại học Bách Khoa Hà Nội,Tổng quan về hệ thống thông tin dưới nước; đồn...,,http://dlib.hust.edu.vn/handle/HUST/22052,Ts-Điện tử viễn thông,,


In [107]:
# hust[hust['Abstract'].map(lambda x: len(x)>100)]
hust=hust[hust['Abstract'].map(lambda x: len(x)>100)]
hust.reset_index(drop=True, inplace=True)
hust.to_excel('Thesis_HUST.xlsx',index=False)

In [118]:
from huggingface_hub import hf_hub_download
import pandas as pd

REPO_ID = "VictorJuiz/thesis"
FILENAME = "Thesis_Foreign.csv"

foreign = pd.read_csv(
    hf_hub_download(repo_id=REPO_ID, filename=FILENAME, repo_type="dataset")
)

In [119]:
print(hust.columns)
print(foreign.columns)

thesis_concat = pd.concat([hust[foreign.columns], foreign[foreign.columns]], axis=0)
thesis_concat.reset_index(drop=True, inplace=True)
# thesis_concat.to_csv('Thesis_concate.csv',index=False)

Index(['Title', 'Link', 'Check', 'Authors', 'Advisor', 'Keywords',
       'Issue Date', 'Publisher', 'Abstract', 'Description', 'URI',
       'Appears in Collection', 'Other Titles', 'Editors'],
      dtype='object')
Index(['Title', 'Link', 'Abstract', 'Keywords', 'Advisor'], dtype='object')


In [123]:
uet=pd.read_excel('Thesis_UET.xlsx')
uet=uet.fillna('')
uet.astype(str)
uet.head()

,Title,Link,Type,Abstract,Keyword,Citation,Author,Advisor,Year,Collection
0,Nghiên cứu gợi ý Api trong mã nguồn Java,https://repository.vnu.edu.vn/handle/VNU_123/1...,Thesis,"Hiện nay, API là một công cụ hữu ích cho các n...","Java, Kỹ thuật phần mềm","Trần, M. C. (2023). Nghiên cứu gợi ý Api trong...","Trần, Mạnh Cường","Võ, Đình Hiếu",2023,UET - Conference Papers
1,Cải tiến phương pháp sinh dữ liệu kiểm thử tự ...,https://repository.vnu.edu.vn/handle/VNU_123/1...,Dissertation,Kiểm thử phần mềm tự động được biết đến như là...,"Ngôn ngữ lập trình, Kiểm thử tự động","Trần, N. H. (2023). Cải tiến phương pháp sinh ...","Trần, Nguyên Hương","Phạm, Ngọc Hùng",2023,UET - Conference Papers
2,Algorithms and Hardware Architectures for high...,https://repository.vnu.edu.vn/handle/VNU_123/1...,Dissertation,This thesis aims to propose efficient solution...,"Mạng thần kinh, Điện tử học","Nguyễn, D. A. (2023). Algorithms and Hardware ...","Nguyễn, Duy Anh","Tran, Xuan Tu\nIacopi, Francesca",2023,UET - Dissertations
3,Nghiên cứu và xây dựng hệ thống cảnh báo trượt...,https://repository.vnu.edu.vn/handle/VNU_123/1...,Dissertation,Ứng dụng mạng cảm biến không dây WSN (Wireless...,Kỹ thuật điện tử ; Mạng cảm biến không dây,"Giản, Q. A. (2023). Nghiên cứu và xây dựng hệ ...","Giản, Quốc Anh","Trần, Đức Tân\nBùi, Tiến Diệu",2023,UET - Dissertations
4,"Nghiên cứu chế tạo hạt nano Ag, Au và nanocomp...",https://repository.vnu.edu.vn/handle/VNU_123/1...,Dissertation,Xây dựng hệ tương tác plasma chất lỏng để chế ...,Vật liệu ; Linh kiện,"Nguyễn, T. T. T. (2023). Nghiên cứu chế tạo hạ...","Nguyễn, Thị Thu Thủy","Nguyễn, Thế Hiện\nĐỗ, Hoàng Tùng",2023,UET - Dissertations


In [129]:
uet[uet['Abstract'].map(lambda x: len(x)>100)]
uet.reset_index(drop=True, inplace=True)
uet.to_excel('Thesis_UET.xlsx',index=False)